# Modeling 1: Clustering

## Anforderungen an Projektumsetzung: Clustering

Falls die Anforderungen auf den eigenen Projektdatensatz nicht anwendbar sind, gibt es folgende Möglichkeiten:
- Datensatz um passende Daten erweitern entweder manuell, per Anspielen aus weiterer Datenquelle oder zur Not per zufälliger Datengenerierung/-simulation
- Datensatz außerhalb des Projekts wählen und Anforderungen daran umsetzen

Anforderungen:
- Führen Sie mit dem Algorithmus Ihrer Wahl eine Clusteranalyse auf Ihren Daten durch.

- Setzen Sie die Parameter des Algorithmus mit Hilfe überwachter oder unüberwachter Evaluationsmethoden. Begründen Sie die Auswahl des Evaluationsalgorithmus.

- Betrachten Sie die gebildeten Cluster. Wie gut sind sie intuitiv? Welche Informationen über Ihren Datensatz ziehen Sie daraus? Leiten sich weitere Schritte der Datenbereinigung oder der Datenaufbereitung ab?

- Können Sie in Ihrem Projektkontext Clustering noch für weitere Zwecke (z.B. Outlier Detection oder Profilerstellung) verwenden? (Dies ist nicht immer der Fall.) Skizzieren Sie ggf. kurz ein mögliches Vorgehen. 

#### Laden der Bibliotheken & des Datensatzes

In [63]:
# Import der benötigten Bibliotheken

import importlib
import init_notebook

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.utils import shuffle

In [64]:
importlib.reload(init_notebook)

dataset = init_notebook.get_final_dataset()

dataset = shuffle(dataset)

# Raucherstatus encoden:

dataset['Raucher_nie'] = np.where(dataset['Raucher_Status'] == 1, 1, 0)
dataset['Raucher_ehem'] = np.where(dataset['Raucher_Status'] == 2, 1, 0)
dataset['Raucher_aktiv'] = np.where(dataset['Raucher_Status'] == 3, 1, 0)

dataset = dataset.drop(columns=['Raucher_Status'])

#### Führen Sie mit dem Algorithmus Ihrer Wahl eine Clusteranalyse auf Ihren Daten durch.

Für das Durchführen der Clusteranalyse werden wir den K-Means Algorithmus verwenden, da wir bisher keine Erfahrungen im Clustering haben.

Ausgewertet werden die Clsuter zunächst basierend auf ihrer Größe, da ein intuitiver Ansatz nach Ähnlichkeit bei der hohen Anzahl an Features nicht möglich ist. 

In [65]:
# Funktion zum clustern des Datasets:

def kmeans_clustering_basic(no_of_clusters, df):
    print('---------------------')
    print('Anzahl Cluster: ' + str(no_of_clusters))
    kmeans_cleaned_dataset = KMeans(n_clusters=no_of_clusters, random_state=0, n_init='auto')
    df['Cluster'] = kmeans_cleaned_dataset.fit_predict(df)

    for i in range(0,no_of_clusters):
        print("Name:", i, "Größe: ", list(kmeans_cleaned_dataset.labels_).count(i))

In [66]:
for i in range(2,15):
    kmeans_clustering_basic(i, dataset)

---------------------
Anzahl Cluster: 2
Name: 0 Größe:  853886
Name: 1 Größe:  137083
---------------------
Anzahl Cluster: 3
Name: 0 Größe:  729233
Name: 1 Größe:  244569
Name: 2 Größe:  17167
---------------------
Anzahl Cluster: 4
Name: 0 Größe:  481597
Name: 1 Größe:  345885
Name: 2 Größe:  152624
Name: 3 Größe:  10863
---------------------
Anzahl Cluster: 5
Name: 0 Größe:  431016
Name: 1 Größe:  218003
Name: 2 Größe:  51160
Name: 3 Größe:  4329
Name: 4 Größe:  286461
---------------------
Anzahl Cluster: 6
Name: 0 Größe:  428441
Name: 1 Größe:  213501
Name: 2 Größe:  283129
Name: 3 Größe:  4212
Name: 4 Größe:  47990
Name: 5 Größe:  13696
---------------------
Anzahl Cluster: 7
Name: 0 Größe:  367275
Name: 1 Größe:  207275
Name: 2 Größe:  252105
Name: 3 Größe:  2774
Name: 4 Größe:  26634
Name: 5 Größe:  12446
Name: 6 Größe:  122460
---------------------
Anzahl Cluster: 8
Name: 0 Größe:  368692
Name: 1 Größe:  200458
Name: 2 Größe:  2506
Name: 3 Größe:  3969
Name: 4 Größe:  246476
N

Die Cluster weißen über alle Iterationen mit einer unterschiedlichen Anzahl an Clustern große Abweichungen in ihrer Größe auf.

In [67]:
# Skaling des Datasets (0 bis 1):

importlib.reload(init_notebook)

dataset_scaled, scaler = init_notebook.minmax_scale_columns(dataset)

dataset_scaled.head()

,Geschlecht,Alter,Größe,Gewicht,Hüftumfang,Systolischer Blutdruck,Diastolischer Blutdruck,Nüchterner Blutzucker,Totale Cholesterin,HDL_Cholesterin,...,Hämoglobin,SGOT_AST,SGOT_ALT,gamma_GTP,Trinker,Body-Mass-Index,Raucher_nie,Raucher_ehem,Raucher_aktiv,Cluster
895977,1.0,0.076923,0.750000,0.391304,0.499646,0.208738,0.281046,0.071342,0.059637,0.066524,...,0.645833,0.026052,0.024048,0.038114,1.0,0.312161,1.0,0.0,0.0,0.000000
252430,0.0,0.615385,0.416667,0.217391,0.510276,0.218447,0.235294,0.099154,0.081245,0.097639,...,0.491667,0.024048,0.016032,0.010030,1.0,0.250452,1.0,0.0,0.0,0.461538
225225,1.0,0.461538,0.666667,0.434783,0.630758,0.276699,0.346405,0.100363,0.047969,0.071888,...,0.645833,0.038076,0.065130,0.051153,0.0,0.405178,0.0,1.0,0.0,0.692308
596639,0.0,0.615385,0.416667,0.260870,0.489015,0.305825,0.313725,0.096735,0.064391,0.059013,...,0.508333,0.014028,0.007014,0.016048,0.0,0.313064,1.0,0.0,0.0,0.923077
934817,1.0,0.461538,0.583333,0.434783,0.574061,0.242718,0.241830,0.099154,0.057908,0.068670,...,0.558333,0.016032,0.012024,0.029087,1.0,0.453341,1.0,0.0,0.0,0.000000


In [68]:
# Clustering des gescaleten Datasets:

for i in range(2,15):
    kmeans_clustering_basic(i, dataset_scaled)

---------------------
Anzahl Cluster: 2
Name: 0 Größe:  602360
Name: 1 Größe:  388609
---------------------
Anzahl Cluster: 3
Name: 0 Größe:  437719
Name: 1 Größe:  388609
Name: 2 Größe:  164641
---------------------
Anzahl Cluster: 4
Name: 0 Größe:  313163
Name: 1 Größe:  388609
Name: 2 Größe:  164641
Name: 3 Größe:  124556
---------------------
Anzahl Cluster: 5
Name: 0 Größe:  313163
Name: 1 Größe:  213789
Name: 2 Größe:  164641
Name: 3 Größe:  124556
Name: 4 Größe:  174820
---------------------
Anzahl Cluster: 6
Name: 0 Größe:  313163
Name: 1 Größe:  124556
Name: 2 Größe:  174820
Name: 3 Größe:  161426
Name: 4 Größe:  52363
Name: 5 Größe:  164641
---------------------
Anzahl Cluster: 7
Name: 0 Größe:  313163
Name: 1 Größe:  52363
Name: 2 Größe:  174820
Name: 3 Größe:  88819
Name: 4 Größe:  161426
Name: 5 Größe:  75822
Name: 6 Größe:  124556
---------------------
Anzahl Cluster: 8
Name: 0 Größe:  313163
Name: 1 Größe:  88819
Name: 2 Größe:  124556
Name: 3 Größe:  120415
Name: 4 Größ

Durch das Scalen des Datasets werden die Differenzen zwischen den einzelnen Cluster kleiner.

Hierbei scheint es, als wären 7 Cluster die optimale Anzahl, da hier die Abweichungen verhältnismäßig klein sind.

#### Setzen Sie die Parameter des Algorithmus mit Hilfe überwachter oder unüberwachter Evaluationsmethoden. Begründen Sie die Auswahl des Evaluationsalgorithmus.

Da in unserem Datensatz keine Zielklassenannotation vorliegt, verwenden wir unüberwachte Evaluationsmethoden zur Untersuchung der Cluster. 

Zudem versuchen wir diese intuitiv über diverse Statistiken (Min, Max, Mean, Median) zu untersuchen.

In [69]:
# Clustering Datasets:

data_scaled_1 = dataset_scaled
data_scaled_2 = dataset_scaled
data_scaled_3 = dataset_scaled
data_scaled_4 = dataset_scaled
data_scaled_5 = dataset_scaled
data_scaled_6 = dataset_scaled
data_scaled_7 = dataset_scaled
data_scaled_8 = dataset_scaled
data_scaled_9 = dataset_scaled
data_scaled_10 = dataset_scaled
data_scaled_11 = dataset_scaled
data_scaled_12 = dataset_scaled
data_scaled_13 = dataset_scaled
data_scaled_14 = dataset_scaled
data_scaled_15 = dataset_scaled
data_scaled_16 = dataset_scaled
data_scaled_17 = dataset_scaled


# Erneutes Laden der Funktionen:

importlib.reload(init_notebook)

<module 'init_notebook' from 'c:\\Users\\SebastianGrab\\OneDrive - DESIGNated Accessoires UG\\Desktop\\DataAnalytics\\Projekt\\Anforderungen\\init_notebook.py'>

In [70]:
init_notebook.cluster_n_print(data_scaled_1, 5, 0, 40)

----------------------------------------
Inertia: 1151640.2558064398
n_clusters= 5
random_state= 0
n_init= 40
----------------------------------------


In [71]:
init_notebook.cluster_analytics(data_scaled_1, 5)

Geschlecht max. - Cluster 0: 1.0
Geschlecht min. - Cluster 0: 0.0
Geschlecht med. - Cluster 0: 1.0
Geschlecht mean - Cluster 0: 0.9249269358748747

Geschlecht max. - Cluster 1: 0.0
Geschlecht min. - Cluster 1: 0.0
Geschlecht med. - Cluster 1: 0.0
Geschlecht mean - Cluster 1: 0.0

Geschlecht max. - Cluster 2: 1.0
Geschlecht min. - Cluster 2: 0.0
Geschlecht med. - Cluster 2: 1.0
Geschlecht mean - Cluster 2: 0.9644544692172002

Geschlecht max. - Cluster 3: 1.0
Geschlecht min. - Cluster 3: 0.0
Geschlecht med. - Cluster 3: 1.0
Geschlecht mean - Cluster 3: 0.9654309017585544

Geschlecht max. - Cluster 4: 0.0
Geschlecht min. - Cluster 4: 0.0
Geschlecht med. - Cluster 4: 0.0
Geschlecht mean - Cluster 4: 0.0

-----------------------------------
Alter max. - Cluster 0: 1.0
Alter min. - Cluster 0: 0.0
Alter med. - Cluster 0: 0.3846153846153846
Alter mean - Cluster 0: 0.39997492320230715

Alter max. - Cluster 1: 1.0
Alter min. - Cluster 1: 0.0
Alter med. - Cluster 1: 0.46153846153846156
Alter mean

Es fällt auf, dass Cluster 0 und besonders Cluster 1 vor allem Nicht-Raucher enthalten.

Währenddessen befinden sich in Cluster 3 vor allem ehemalige Raucher und in Cluster 2 sowie teilweise in Cluster 3 aktive Raucher.

Logs dazu: Logs 4 Cluster: [Logs_DataScaled1](Logs/5_Clustering/Logs_DataScaled1.txt)

Dieses Muster könnte sich eventuell auch bei einer geringeren Cluster-Anzahl aufzeigen. Folgend wird dies überprüft.

In [72]:
init_notebook.cluster_n_print(data_scaled_2, 4, 0, 40)

----------------------------------------
Inertia: 677653.3436896177
n_clusters= 4
random_state= 0
n_init= 40
----------------------------------------


In [73]:
init_notebook.cluster_analytics(data_scaled_2, 4)

Geschlecht max. - Cluster 0: 0.0
Geschlecht min. - Cluster 0: 0.0
Geschlecht med. - Cluster 0: 0.0
Geschlecht mean - Cluster 0: 0.0

Geschlecht max. - Cluster 1: 1.0
Geschlecht min. - Cluster 1: 0.0
Geschlecht med. - Cluster 1: 1.0
Geschlecht mean - Cluster 1: 0.9649627304390266

Geschlecht max. - Cluster 2: 0.0
Geschlecht min. - Cluster 2: 0.0
Geschlecht med. - Cluster 2: 0.0
Geschlecht mean - Cluster 2: 0.0

Geschlecht max. - Cluster 3: 1.0
Geschlecht min. - Cluster 3: 0.0
Geschlecht med. - Cluster 3: 1.0
Geschlecht mean - Cluster 3: 0.9249269358748747

-----------------------------------
Alter max. - Cluster 0: 1.0
Alter min. - Cluster 0: 0.0
Alter med. - Cluster 0: 0.46153846153846156
Alter mean - Cluster 0: 0.4289032458864384

Alter max. - Cluster 1: 1.0
Alter min. - Cluster 1: 0.0
Alter med. - Cluster 1: 0.3846153846153846
Alter mean - Cluster 1: 0.40714115675570517

Alter max. - Cluster 2: 1.0
Alter min. - Cluster 2: 0.0
Alter med. - Cluster 2: 0.5384615384615385
Alter mean - Cl

In [74]:
init_notebook.cluster_n_print(data_scaled_3, 3, 0, 40)

----------------------------------------
Inertia: 771340.4715367915
n_clusters= 3
random_state= 0
n_init= 40
----------------------------------------


In [75]:
init_notebook.cluster_analytics(data_scaled_3, 3)

Geschlecht max. - Cluster 0: 0.0
Geschlecht min. - Cluster 0: 0.0
Geschlecht med. - Cluster 0: 0.0
Geschlecht mean - Cluster 0: 0.0

Geschlecht max. - Cluster 1: 1.0
Geschlecht min. - Cluster 1: 0.0
Geschlecht med. - Cluster 1: 1.0
Geschlecht mean - Cluster 1: 0.5169840445441277

Geschlecht max. - Cluster 2: 1.0
Geschlecht min. - Cluster 2: 0.0
Geschlecht med. - Cluster 2: 1.0
Geschlecht mean - Cluster 2: 0.9649627304390266

-----------------------------------
Alter max. - Cluster 0: 1.0
Alter min. - Cluster 0: 0.0
Alter med. - Cluster 0: 0.46153846153846156
Alter mean - Cluster 0: 0.4289032458864384

Alter max. - Cluster 1: 1.0
Alter min. - Cluster 1: 0.0
Alter med. - Cluster 1: 0.3846153846153846
Alter mean - Cluster 1: 0.43996856538384527

Alter max. - Cluster 2: 1.0
Alter min. - Cluster 2: 0.0
Alter med. - Cluster 2: 0.3846153846153846
Alter mean - Cluster 2: 0.40714115675570517

-----------------------------------
Größe max. - Cluster 0: 0.75
Größe min. - Cluster 0: 0.0
Größe med.

Obwohl die Inertia-Werte hier niedriger sind, lässt sich das zuvor erkannte Schema beim Training mit 4 und 3 Clustern nicht wiederfinden.

Logs 4 Cluster: [Logs_DataScaled2](Logs/5_Clustering/Logs_DataScaled2.txt)

Logs 3 Cluster: [Logs_DataScaled2](Logs/5_Clustering/Logs_DataScaled3.txt)

Deshalb wird zunächst die Aufteilung in 5 Cluster beibehalten.

Folgend wird mit unterscheidlichen Random-States trainiert:

In [76]:
init_notebook.cluster_n_print(data_scaled_4, 5, 10, 40)
init_notebook.cluster_n_print(data_scaled_5, 5, 20, 40)
init_notebook.cluster_n_print(data_scaled_6, 5, 30, 40)
init_notebook.cluster_n_print(data_scaled_7, 5, 40, 40)

----------------------------------------
Inertia: 398424.5775460484
n_clusters= 5
random_state= 10
n_init= 40
----------------------------------------
----------------------------------------
Inertia: 323536.0531321154
n_clusters= 5
random_state= 20
n_init= 40
----------------------------------------
----------------------------------------
Inertia: 323536.0531321154
n_clusters= 5
random_state= 30
n_init= 40
----------------------------------------
----------------------------------------
Inertia: 323536.0531321154
n_clusters= 5
random_state= 40
n_init= 40
----------------------------------------


Erkenntnis: 

Bis auf das trainierte Modell mit einem Random-State von 10 gibt es nur minimale Veränderungen des Inertia-Wertes. 
Es zeigt sich, dass eine Veränderung der Random States in diesem Ausmaß keine große Veränderung hervorbringt.

Deshalb werden diese in der folgenden Zelle stärker variiert. 

In [77]:
data_scaled_8, kmeans_scaled_8 = init_notebook.cluster_n_print_ver2(data_scaled_8, 5, 150, 40)
data_scaled_9, kmeans_scaled_9 = init_notebook.cluster_n_print_ver2(data_scaled_9, 5, 1000, 40)
data_scaled_10, kmeans_scaled_10 = init_notebook.cluster_n_print_ver2(data_scaled_10, 6, 0, 40)
data_scaled_11, kmeans_scaled_11 = init_notebook.cluster_n_print_ver2(data_scaled_11, 4, 0, 40)
data_scaled_12, kmeans_scaled_12 = init_notebook.cluster_n_print_ver2(data_scaled_12, 3, 0, 40)

----------------------------------------


Inertia: 323536.0531321154
n_clusters= 5
random_state= 150
n_init= 40

Clustergrößen: 
Name: 0 Größe:  152838
Name: 1 Größe:  285706
Name: 2 Größe:  213789
Name: 3 Größe:  174864
Name: 4 Größe:  163772


----------------------------------------
----------------------------------------


Inertia: 323536.0531321154
n_clusters= 5
random_state= 1000
n_init= 40

Clustergrößen: 
Name: 0 Größe:  163772
Name: 1 Größe:  152838
Name: 2 Größe:  213789
Name: 3 Größe:  285706
Name: 4 Größe:  174864


----------------------------------------
----------------------------------------


Inertia: 275089.3197384917
n_clusters= 6
random_state= 0
n_init= 40

Clustergrößen: 
Name: 0 Größe:  285706
Name: 1 Größe:  87950
Name: 2 Größe:  213789
Name: 3 Größe:  174864
Name: 4 Größe:  152838
Name: 5 Größe:  75822


----------------------------------------
----------------------------------------


Inertia: 464241.3223973883
n_clusters= 4
random_state= 0
n_init= 40

Clus

In [78]:
init_notebook.cluster_analytics(data_scaled_10, 6)

Geschlecht max. - Cluster 0: 0.0
Geschlecht min. - Cluster 0: 0.0
Geschlecht med. - Cluster 0: 0.0
Geschlecht mean - Cluster 0: 0.0

Geschlecht max. - Cluster 1: 1.0
Geschlecht min. - Cluster 1: 0.0
Geschlecht med. - Cluster 1: 1.0
Geschlecht mean - Cluster 1: 0.8143307461266245

Geschlecht max. - Cluster 2: 1.0
Geschlecht min. - Cluster 2: 0.0
Geschlecht med. - Cluster 2: 1.0
Geschlecht mean - Cluster 2: 0.9240699942466638

Geschlecht max. - Cluster 3: nan
Geschlecht min. - Cluster 3: nan
Geschlecht med. - Cluster 3: nan
Geschlecht mean - Cluster 3: nan

Geschlecht max. - Cluster 4: nan
Geschlecht min. - Cluster 4: nan
Geschlecht med. - Cluster 4: nan
Geschlecht mean - Cluster 4: nan

Geschlecht max. - Cluster 5: nan
Geschlecht min. - Cluster 5: nan
Geschlecht med. - Cluster 5: nan
Geschlecht mean - Cluster 5: nan

-----------------------------------
Alter max. - Cluster 0: 1.0
Alter min. - Cluster 0: 0.0
Alter med. - Cluster 0: 0.46153846153846156
Alter mean - Cluster 0: 0.4966931739

Das weitere Variieren der Random-States zeigt keinen wesentlichen Unterschied zu kleineren Random-States.

Es zeigte sich zudem erneut, dass eine Verkleinerung der Cluster-Anzahl keine positiven Auswirkungen hat.

Wird die Anzahl jedoch auf 6 erhöht, zeigt sich, dass sich auch hier das bereits bekannte Schema zur Verteilung der Raucherstati ergibt.

Hierbei befinden sich Nicht-Raucher vor allem in Cluster 0, ehemalige Raucher vor allem in Cluster 1 und Raucher ausschließlich in Cluster 2.
Cluster 3, 4 & 5 weisen hier jedoch lediglich 'nan' Werte auf.

Logs 6 Cluster: [Logs_DataScaled10](Logs/5_Clustering/Logs_DataScaled10.txt)

Deshalb folgend erneut eine Erhöhung der Cluster-Anzahl:

In [79]:
data_scaled_13, kmeans_scaled_13 = init_notebook.cluster_n_print_ver2(data_scaled_13, 6, 20, 40)
data_scaled_14, kmeans_scaled_14 = init_notebook.cluster_n_print_ver2(data_scaled_14, 7, 20, 40)
data_scaled_15, kmeans_scaled_15 = init_notebook.cluster_n_print_ver2(data_scaled_15, 8, 20, 40)
data_scaled_16, kmeans_scaled_16 = init_notebook.cluster_n_print_ver2(data_scaled_16, 9, 20, 40)
data_scaled_17, kmeans_scaled_17 = init_notebook.cluster_n_print_ver2(data_scaled_17, 10, 20, 40)

----------------------------------------


Inertia: 227020.39349024478
n_clusters= 6
random_state= 20
n_init= 40

Clustergrößen: 
Name: 0 Größe:  174864
Name: 1 Größe:  313119
Name: 2 Größe:  213789
Name: 3 Größe:  124556
Name: 4 Größe:  75822
Name: 5 Größe:  88819


----------------------------------------
----------------------------------------


Inertia: 155970.3354111043
n_clusters= 7
random_state= 20
n_init= 40

Clustergrößen: 
Name: 0 Größe:  313119
Name: 1 Größe:  88819
Name: 2 Größe:  52363
Name: 3 Größe:  174864
Name: 4 Größe:  124556
Name: 5 Größe:  75822
Name: 6 Größe:  161426


----------------------------------------
----------------------------------------


Inertia: 118015.27047124263
n_clusters= 8
random_state= 20
n_init= 40

Clustergrößen: 
Name: 0 Größe:  161426
Name: 1 Größe:  313119
Name: 2 Größe:  120415
Name: 3 Größe:  88819
Name: 4 Größe:  124556
Name: 5 Größe:  75822
Name: 6 Größe:  52363
Name: 7 Größe:  54449


----------------------------------------
--------

In [81]:
init_notebook.cluster_analytics(data_scaled_14, 7)

Geschlecht max. - Cluster 0: 0.0
Geschlecht min. - Cluster 0: 0.0
Geschlecht med. - Cluster 0: 0.0
Geschlecht mean - Cluster 0: 0.0

Geschlecht max. - Cluster 1: 0.0
Geschlecht min. - Cluster 1: 0.0
Geschlecht med. - Cluster 1: 0.0
Geschlecht mean - Cluster 1: 0.0

Geschlecht max. - Cluster 2: 0.0
Geschlecht min. - Cluster 2: 0.0
Geschlecht med. - Cluster 2: 0.0
Geschlecht mean - Cluster 2: 0.0

Geschlecht max. - Cluster 3: 1.0
Geschlecht min. - Cluster 3: 1.0
Geschlecht med. - Cluster 3: 1.0
Geschlecht mean - Cluster 3: 1.0

Geschlecht max. - Cluster 4: 0.0
Geschlecht min. - Cluster 4: 0.0
Geschlecht med. - Cluster 4: 0.0
Geschlecht mean - Cluster 4: 0.0

Geschlecht max. - Cluster 5: 1.0
Geschlecht min. - Cluster 5: 1.0
Geschlecht med. - Cluster 5: 1.0
Geschlecht mean - Cluster 5: 1.0

Geschlecht max. - Cluster 6: 1.0
Geschlecht min. - Cluster 6: 0.0
Geschlecht med. - Cluster 6: 1.0
Geschlecht mean - Cluster 6: 0.9461113648631815

-----------------------------------
Alter max. - Clust

Bereits beim Clustering mit 7 Clustern zeigt sich, dass sich nicht nur die unterschiedlichen Raucherstati, sondern ebenso der Trinkerstatus und das Geschlecht nahezu perfekt auf die unterschiedlichen Cluster aufteilen.

Logs 7 Cluster: [Logs_DataScaled14](Logs/5_Clustering/Logs_DataScaled14.txt)

Diese Tendenz bestätigt sich zudem bei noch mehr Clustern (8, 9 oder 10), verbessert das Ergebnis jedoch nicht wesentlich.

#### Betrachten Sie die gebildeten Cluster. Wie gut sind sie intuitiv? Welche Informationen über Ihren Datensatz ziehen Sie daraus? Leiten sich weitere Schritte der Datenbereinigung oder der Datenaufbereitung ab?

Die Cluster sind auf Grund der großen Feature-Anzahl nur sehr schwer intuitiv interpretierbar. Eine Hauptkomponentenanalyse wird sich vermutlich lohnen. 

Dennoch konnte erkannt werden, dass sich das Clustering vor allem auf den Raucherstatus, jedoch ebenso auf den Trinkerstatus und das Geschlecht beziehen.
Durch die vielen Trainingsversuche konnte festgestellt werden, dass das Clustern mit 7 Clustern diese Features besonders gut in unterschiedliche Cluster aufteilen kann.

Es muss sich gefragt werden, weshalb der Algoritmus genau diese Features so geclustert hat. Vermutlich sind Features mit den Ausprägungen 1 & 0 einfacher zuordenbar als Features mit einer größeren Ausprägungsvarianz.

Es ich wichtig, bei weiteren Analysen die jeweiligen Ausprägungsvarianzen der einzelnen Features zu betrachten und diese zu hinterfragen.

#### Können Sie in Ihrem Projektkontext Clustering noch für weitere Zwecke (z.B. Outlier Detection oder Profilerstellung) verwenden? (Dies ist nicht immer der Fall.) Skizzieren Sie ggf. kurz ein mögliches Vorgehen. 

Die Clusteranalyse könnte gegebenenfalls dabei helfen, fehlerhafte Daten zu finden. 
Sollte beispielsweise die Messung eines Wertes logisch gar nicht zu den anderen Werten passen, ließe sich aufzeigen, dass die Messung fehlerhaft war.  

Bei Abfragen wie der des Raucher- oder Trinkerstatus besteht die Gefahr, dass ein Proband nicht wahrheitsgemäß antwortet, um ggf. besser eingestuft zu werden. 
Anhand einer Clusteranalyse könnte dies beispielsweise durch den Hämoglobinwert (korreliert mit Raucher_Status postiv als auich mit Nichtraucher_Status negativ) identifiziert werden. 

Dazu könnte man alle korrelierenden Faktoren (Geschlecht, Gewicht, Größe, Hüftumfang, Hamoglobin) extrahieren und Clustern lassen. 
Dadurch wäre supervised learning möglich und man wäre in der Lage, die Aussage eines Probanden bewerten zu können.

Es muss jedoch berücksichtigt werden, dass die Inerpretation von Befragungswerten (wie beispielsweise des Raucher- oder Trinkerstatus) im Versicherungskontext stets hinterfragt werden sollte.